In [18]:
import pandas as pd
import numpy as np
import csv
import sys
import os
from pathlib import Path

In [35]:
path_ENRO_bisents = os.path.join(os.pardir, "data/00-raw/EN-RO-bisentences.txt")
#quoting is to correctly include quote in quote strings as one field when reading the file
df=pd.read_csv(path_ENRO_bisents, sep='\t', names=['english', 'romanian'], quoting=csv.QUOTE_NONE)

The raw file (df.shape=(2668811, 2)) contains lots of duplicates. Here are some of them:

In [36]:
df_duplcates_sorted=df.groupby(df.columns.tolist(),as_index=False).size().sort_values(by='size',ascending=[False])[:8]
df_duplcates_sorted

,english,romanian,size
174855,Amendment,Amendamentul,59392
872241,–,–,33074
55777,",",",",31713
596108,Text proposed by the Commission,Textul propus de Comisie,24069
433286,Justification,Justificare,18142
93126,1.,1.,14492
561622,Rule,Articolul,11383
528323,Proposal for a regulation,Propunere de regulament,11349


Removing suplicates and NaN values

In [37]:
df=df.dropna()
df=df.drop_duplicates()
df.shape

(902303, 2)

In [38]:
pd.set_option('display.max_colwidth', None)
df.iloc[591524:591529]

,english,romanian
1970266,This recommendation is one of the key elements of the report.,Această recomandare este unul dintre elementele esenţiale ale raportului.
1970267,"Particular attention should be paid to the dangers of illegal, unreported and unregulated fishing.","Ar trebui acordată o atenţie deosebită pericolelor pescuitului ilegal, nedeclarat şi nereglementat."
1970268,Both EU and non-EU countries in the Black Sea basin should take strict measures to fight IUU fishing.,"Atât ţările membre UE, cât şi ţările terţe din bazinul Mării Negre ar trebui să ia măsuri stricte pentru combaterea pescuitului INN."
1970269,This could only be achieved through the active cooperation of the European Commission and the Black Sea countries.,Acest lucru nu poate fi realizat decât prin cooperarea activă dintre Comisia Europeană şi ţările de la Marea Neagră.
1970270,The Committee on Fisheries has repeatedly stressed the importance of taking into account regional specificities when shaping future policies.,Comisia pentru pescuit a subliniat în repetate rânduri importanţa luării în considerare a specificităţilor regionale la formularea politicilor viitoare.


Found untranslated sentence:

In [39]:
contains_untranslated = df[df['romanian'].str.contains('Pursuant to this differentiation the Conference')]
contains_untranslated

,english,romanian
1458024,Pursuant to this differentiation the Conference of Presidents also decided to refer to the Committee on Constitutional Affairs new items requiring changes to Parliament's Rules of Procedure.,Pursuant to this differentiation the Conference of Presidents also decided to refer to the Committee on Constitutional Affairs new items requiring changes to Regulamentul de procedură al Parlamentului.


If we remove also alternative translations we are left with 787896 sentence pairs

In [41]:
df_no_ro_alternative=df.drop_duplicates(subset='english')
df_no_ro_alternative.shape

(820436, 2)

In [44]:
df_no_en_alternative=df.drop_duplicates(subset='romanian')
df_no_en_alternative.shape

df_no_alternatives=df_no_ro_alternative.drop_duplicates(subset='romanian')
df_no_alternatives.shape

(787895, 2)

In [45]:
df=df_no_alternatives
df.shape

(787895, 2)

In [46]:
source_dataset=df.iloc[:,0]
target_dataset=df.iloc[:,1]

print(source_dataset.size==target_dataset.size, source_dataset.size)

True 787895


In [ ]:
Example of freely translated sentence:

In [50]:
contains_freely=df[df.iloc[:, 1].str.contains('joc de cuvinte')]
contains_freely

,english,romanian
322678,"We ""pass"" arguments among ourselves, exchange views and experience in order to achieve a common ""goal"".","Ne «pasăm» argumentele de la unul la altul, facem schimburi de opinii şi de experienţe pentru a obţine un «obiectiv» [joc de cuvinte în limba engleză, «goal» însemnând în acelaşi timp «obiectiv» şi «gol» n.r.]."


In [ ]:
path_source_raw= (base_path / "../data/01-intermediate/01-01-fulldata/EN_raw.txt").resolve()
path_target_raw= (base_path / "../data/01-intermediate/01-01-fulldata/RO_raw.txt").resolve()

source_dataset.to_csv (path_source_raw , index = None, header = False)
target_dataset.to_csv (path_target_raw , index = None, header = False)


In [ ]:
! pip install -U sacremoses

     |████████████████████████████████| 901kB 33.3MB/s 


In [ ]:
source_file= "/content/drive/MyDrive/data_en_ro/dcep_full/clean_dcep.ro-en.en.txt"
target_file= "/content/drive/MyDrive/data_en_ro/dcep_full/clean_dcep.ro-en.ro.txt"
tok_source_file = source_file+".tok"
tok_target_file=target_file+".tok"

#try spacy tokenizer?
! sacremoses -l "en" -j 8 tokenize < $source_file > $tok_source_file
! sacremoses -l "ro" -j 8 tokenize < $target_file > $tok_target_file

! head $source_file*
! head $target_file*

! wc -l $source_file*
! wc -l $target_file*

In [ ]:
import pandas as pd

source_file= "/content/drive/MyDrive/data_en_ro/dcep_full/clean_dcep.ro-en.en.txt.tok"
target_file= "/content/drive/MyDrive/data_en_ro/dcep_full/clean_dcep.ro-en.ro.txt.tok"

source=[]
target=[]

source_test=[]
target_test=[]
skip_lines=[] #collect the line numbers of the source portion to skip the same lines for the taret portion

# a manual alternative to train_test_split from scikit-learn
with open(source_file) as f:
  for i, line in enumerate(f):
    #splitting 20% of training into test
    if i % 5 == 0:
      print(i)
      source_test.append(line.strip())
      skip_lines.append(i)
    else:
      print(i)
      source.append(line.strip())

with open(target_file) as f:
  for j, line in enumerate(f):
    if j not in skip_lines:
      print(j)
      target.append(line.strip())
    else:
      print(j)
      target_test.append(line.strip())

print('Loaded data and skipped {}/{} lines since contained in test set. '.format(len(skip_lines), i))

In [ ]:
df= pd.DataFrame(zip(source, target), columns=['english', 'romanian'])
test= pd.DataFrame(zip(source_test, target_test), columns=['english', 'romanian'])

print("train...")
print(df.head(5))

print("test...")
print(test.head(5))

train...
                                                                                                english                                                                                                       romanian
0                                                     for Question Time at the part-session in May 2007                                            pentru timpul afectat întrebărilor din perioada de sesiune mai 2007
1                                                        pursuant to Rule 109 of the Rules of Procedure                                                 în conformitate cu articolul 109 din Regulamentul de procedură
2                                                                                                    by                                                                                                             de
3                                                                                 Roberta Alma Anastase                            

In [ ]:
seed=42

df=df.sample(frac=1, random_state=seed).reset_index(drop=True)
df.head()

,english,romanian
0,"&quot; H. whereas better synergy in future European energy technology research can only stimulate sustainable economic growth , contribute to the comparative advantages of the European economy , improve employment and thus help achieve the objectives of the Lisbon strategy and combat climate change , &quot;","&quot; H. întrucât o mai bună sinergie în privința cercetării europene în domeniul tehnologiilor energetice ale viitorului nu poate decât să încurajeze creșterea economică durabilă , să contribuie la avantajul comparativ al economiei europene , să amelioreze ocuparea forței de muncă și să permită , astfel , atingerea obiectivelor strategiei de la Lisabona și combaterea schimbărilor climatice ; &quot;"
1,"&quot; – a proposal to be put forward by 2013 on the &apos; Blue Belt &apos; , to facilitate the formalities for ships operating between EU ports and to develop the potential of motorways of the sea by establishing a genuine single market for intra-EU maritime transport in accordance with existing environmental and nature conservation legislation ; &quot;","&quot; – o propunere , prezentată până în 2013 , referitoare la așa-numita „ centură albastră ” , pentru simplificarea formalităților pentru navele care operează între porturile din UE și pentru dezvoltarea potențialului autostrăzilor maritime prin realizarea unei piețe unice reale pentru transportul maritim în interiorul UE în conformitate cu legislația existentă în domeniul mediului și al conservării naturii ; &quot;"
2,Report on the future European international investment policy &#91; 2010 / 2203 ( INI ) &#93; - Committee on International Trade .,Raport referitor la viitoarea politică europeană în domeniul investițiilor internaționale &#91; 2010 / 2203 ( INI ) &#93; - Comisia pentru comerţ internaţional .
3,"&quot; Expresses its strong concern about the EU-Morocco agreement ; points out that , while European markets have opened up almost completely to imports from Morocco , some agricultural products are still subject to quotas on exports from the EU , including important products such as pomaceous fruits ; &quot;","&quot; îşi exprimă profunda îngrijorare cu privire la acordul UE-Maroc ; subliniază că , în timp ce pieţele europene sunt acum aproape complet deschise importurilor din Maroc , unele produse agricole , printre care se numără şi unele produse importante precum fructele pomacee , încă sunt supuse contingentelor pentru exporturi din UE ; &quot;"
4,"&quot; We &quot; &quot; pass &quot; &quot; arguments among ourselves , exchange views and experience in order to achieve a common &quot; &quot; goal &quot; &quot; . &quot;","&quot; Ne « pasăm » argumentele de la unul la altul , facem schimburi de opinii şi de experienţe pentru a obţine un « obiectiv » &#91; joc de cuvinte în limba engleză , « goal » însemnând în acelaşi timp « obiectiv » şi « gol » n.r. &#93; . &quot;"


In [ ]:
pd.set_option('display.max_colwidth', None)
df.iloc[3:4]

,english,romanian
3,"&quot; Expresses its strong concern about the EU-Morocco agreement ; points out that , while European markets have opened up almost completely to imports from Morocco , some agricultural products are still subject to quotas on exports from the EU , including important products such as pomaceous fruits ; &quot;","&quot; îşi exprimă profunda îngrijorare cu privire la acordul UE-Maroc ; subliniază că , în timp ce pieţele europene sunt acum aproape complet deschise importurilor din Maroc , unele produse agricole , printre care se numără şi unele produse importante precum fructele pomacee , încă sunt supuse contingentelor pentru exporturi din UE ; &quot;"


In [ ]:
print(len(df.index),len(test.index))

630316 157580


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_full

/content/drive/MyDrive/data_en_ro/dcep_full


In [ ]:
#split btw train and dev
import csv
lc=False 
num_dev_patterns= 10000

if lc:
  df['english']=df['english'].str.lower()
  df['romanian']=df['romanian'].str.lower()
  test['romanian']=test['romanian'].str.lower()
  test['english']=test['english'].str.lower()

#dev set
dev= df.tail(num_dev_patterns)
#training set
stripped=df.drop(df.tail(num_dev_patterns).index)

with open("clean_train.en", "w") as src_file, open("clean_train.ro", "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(str(row["english"])+"\n")
    trg_file.write(str(row["romanian"])+"\n")

with open("clean_dev.en", "w") as src_file, open("clean_dev.ro", "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["english"]+"\n")
    trg_file.write(row["romanian"]+"\n")

with open("clean_test.en", "w") as src_file, open("clean_test.ro", "w") as trg_file:
  for index, row in test.iterrows():
    src_file.write(row["english"]+"\n")
    trg_file.write(row["romanian"]+"\n")

In [ ]:
! head -10 /content/drive/MyDrive/data_en_ro/dcep_full/clean_test.ro

ÎNTREBARE ORALĂ H-0336 / 07
Consiliului
ÎNTREBARE ORALĂ H-0431 / 07
Comisia Europeană şi-a stabilit ca obiectiv reducerea obligaţiilor administrative inutile impuse prin legislaţia UE asupra mediului de afaceri .
ÎNTREBARE ORALĂ H-0513 / 07
Comisia Europeană a aprobat în 2004 o listă de 30 de proiecte prioritare pentru Reţeaua Europeană de Transport în care şi noile state membre sunt incluse .
Subiect : Abandonul şcolar
&quot; În contextul creşterii productivităţii muncii , este nevoie însă ca forţa de muncă să fie tot mai bine pregătită . &quot;
&quot; Având în vedere poziţia adoptată de unele state din regiune privind prioritatea acordată de acestea proiectului Nabucco , aş dori să întreb Consiliul Uniunii Europene care sunt măsurile pe care le are în vedere pentru accelerarea realizării acestui proiect deosebit de important atât pentru statele tranzitate , cât şi pentru aprovizionarea cu gaz a Uniunii Europene şi care sunt termenele de realizare avute în vedere ? &quot;
Ce îşi propu

In [ ]:
df_clean_test_ro = pd.read_csv("/content/drive/MyDrive/data_en_ro/dcep_full/clean_test.ro", sep='\n', header=None)
df_clean_test_en = pd.read_csv("/content/drive/MyDrive/data_en_ro/dcep_full/clean_test.en", sep='\n', header=None)


contain_values = df_clean_test_ro[df_clean_test_ro.iloc[:,0].str.contains('Pursuant to this differentiation the Conference')]

In [ ]:
print(contain_values)

                                                       0
65096  Pursuant to this differentiation the Conferenc...


In [ ]:
pd.set_option('display.max_colwidth', None)

print(df_clean_test_ro.iloc[65096])
print(df_clean_test_en.iloc[65096])

0    Pursuant to this differentiation the Conference of Presidents also decided to refer to the Committee on Constitutional Affairs new items requiring changes to Regulamentul de procedură al Parlamentului .
Name: 65096, dtype: object
0    Pursuant to this differentiation the Conference of Presidents also decided to refer to the Committee on Constitutional Affairs new items requiring changes to Parliament &apos;s Rules of Procedure .
Name: 65096, dtype: object


In [ ]:
df_clean_train_en = pd.read_csv("/content/drive/MyDrive/data_en_ro/dcep_full/clean_train.en", sep='\n', header=None)

contain_values = df_clean_dev_en[df_clean_dev_en.iloc[:,0].str.contains('Clear European')]
print(df_clean_train_en.iloc[:20])

                                                                                                                                                                                                                                                                                                                                                                        0
0                                                    &quot; H. whereas better synergy in future European energy technology research can only stimulate sustainable economic growth , contribute to the comparative advantages of the European economy , improve employment and thus help achieve the objectives of the Lisbon strategy and combat climate change , &quot;
1   &quot; – a proposal to be put forward by 2013 on the &apos; Blue Belt &apos; , to facilitate the formalities for ships operating between EU ports and to develop the potential of motorways of the sea by establishing a genuine single market for intra-EU maritime transport i

In [ ]:
df_clean_train_ro = pd.read_csv("/content/drive/MyDrive/data_en_ro/dcep_full/clean_train.ro", sep='\n', header=None)

contain_values = df_clean_train_ro[df_clean_train_ro.iloc[:,0].str.contains('european clar')]
print(contain_values)
print(df_clean_train_ro.iloc[:20])

                                                                                 0
401777  – un cadru legislativ european clar care să protejeze serviciile publice ;
                                                                                                                                                                                                                                                                                                                                                                                                                                                                     0
0                                                  &quot; H. întrucât o mai bună sinergie în privința cercetării europene în domeniul tehnologiilor energetice ale viitorului nu poate decât să încurajeze creșterea economică durabilă , să contribuie la avantajul comparativ al economiei europene , să amelioreze ocuparea forței de muncă și să permită , astfel , atingerea obiective